In [1]:
from time import strftime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics, decomposition, neighbors
print strftime("%Y-%m-%d %X"), "clock set"

2015-05-03 11:01:09 clock set


In [2]:
# load X_given and y_given
with open('../data/train.csv') as f:
    given_csv = f.readlines()
    # number of columns in each row is 1 + 28^2 = 1 + 784
    given_csv = [np.array(map(int, x.split(','))) for x in given_csv[1:]]
    X_given_all = [x[1:] for x in given_csv]
    y_given_all = [x[0] for x in given_csv]
print strftime("%Y-%m-%d %X"), "given data loaded"

# load X_test
with open('../data/test.csv') as f:
    test_csv = f.readlines()
    # number of columns in each row is 28^2 = 784
    X_test = [np.array(map(int, x.split(','))) for x in test_csv[1:]]
print strftime("%Y-%m-%d %X"), "testing data loaded"

print len(X_given_all)

2015-05-03 11:01:40 given data loaded
2015-05-03 11:02:03 testing data loaded
42000


In [3]:
X_given = X_given_all[0:] # reduce data for tests?
y_given = y_given_all[0:]
print len(X_given), 'out of', len(X_given_all), 'givens loaded'

X_train, X_validate, y_train, y_validate = cross_validation.train_test_split(X_given, y_given, test_size=0, random_state=0)
print strftime("%Y-%m-%d %X"), "cross-validation loaded"

42000 out of 42000 givens loaded
2015-05-03 11:02:03 cross-validation loaded


In [4]:
def appendAllTranslations(X_datum, y_datum, X_list, y_list):
    firstRow, lastRow = 0, 27
    firstColumn, lastColumn = 0, 27
    X_datum = np.reshape(X_datum, [28, 28])
    
    while all(X_datum[firstRow, :] == 0):
        firstRow += 1
    while all(X_datum[lastRow, :] == 0):
        lastRow -= 1
    while all(X_datum[:, firstColumn] == 0):
        firstColumn += 1
    while all(X_datum[:, lastColumn] == 0):
        lastColumn -= 1
    
    center = X_datum[firstRow:lastRow + 1, firstColumn:lastColumn + 1]
    height = lastRow - firstRow + 1
    width = lastColumn - firstColumn + 1
    heightToPad = 28 - height
    widthToPad = 28 - width
    
    for ii in xrange(1, heightToPad): # leave one white pixel on the edge
        for jj in xrange(1, widthToPad):
            X_list.append(np.reshape(np.pad(center, ((ii, heightToPad - ii), (jj, widthToPad - jj)), 'constant'), -1))
            y_list.append(y_datum)
    return

# def appendAllTranslations(X_datum, y_datum, X_list, y_list):
#     firstRow, lastRow = 0, 27
#     firstColumn, lastColumn = 0, 27
    
#     X_list.append(X_datum)
#     y_list.append(y_datum)
    
#     X_datum = np.reshape(X_datum, [28, 28])
    
#     while all(X_datum[firstRow, :] == 0):
#         firstRow += 1
#     while all(X_datum[lastRow, :] == 0):
#         lastRow -= 1
#     while all(X_datum[:, firstColumn] == 0):
#         firstColumn += 1
#     while all(X_datum[:, lastColumn] == 0):
#         lastColumn -= 1

#     if firstRow > 1:
#         X_list.append(np.reshape(np.pad(X_datum[1:, :], ((0, 1), (0, 0)), 'constant'), -1))
#         y_list.append(y_datum)
#         if firstRow > 2:
#             X_list.append(np.reshape(np.pad(X_datum[2:, :], ((0, 2), (0, 0)), 'constant'), -1))
#             y_list.append(y_datum)
#             if firstRow > 3:
#                 X_list.append(np.reshape(np.pad(X_datum[3:, :], ((0, 3), (0, 0)), 'constant'), -1))
#                 y_list.append(y_datum)
#     if lastRow < 27:
#         X_list.append(np.reshape(np.pad(X_datum[:27, :], ((1, 0), (0, 0)), 'constant'), -1))
#         y_list.append(y_datum)
#         if lastRow < 26:
#             X_list.append(np.reshape(np.pad(X_datum[:26, :], ((2, 0), (0, 0)), 'constant'), -1))
#             y_list.append(y_datum)
#             if lastRow < 25:
#                 X_list.append(np.reshape(np.pad(X_datum[:25, :], ((3, 0), (0, 0)), 'constant'), -1))
#                 y_list.append(y_datum)
#     if firstColumn > 1:
#         X_list.append(np.reshape(np.pad(X_datum[:, 1:], ((0, 0), (0, 1)), 'constant'), -1))
#         y_list.append(y_datum)
#         if firstColumn > 2:
#             X_list.append(np.reshape(np.pad(X_datum[:, 2:], ((0, 0), (0, 2)), 'constant'), -1))
#             y_list.append(y_datum)
#             if firstColumn > 3:
#                 X_list.append(np.reshape(np.pad(X_datum[:, 3:], ((0, 0), (0, 3)), 'constant'), -1))
#                 y_list.append(y_datum)
#     if lastColumn < 27:
#         X_list.append(np.reshape(np.pad(X_datum[:, :27], ((0, 0), (1, 0)), 'constant'), -1))
#         y_list.append(y_datum)
#         if lastColumn < 26:
#             X_list.append(np.reshape(np.pad(X_datum[:, :26], ((0, 0), (2, 0)), 'constant'), -1))
#             y_list.append(y_datum)
#             if lastColumn < 25:
#                 X_list.append(np.reshape(np.pad(X_datum[:, :25], ((0, 0), (3, 0)), 'constant'), -1))
#                 y_list.append(y_datum)
#     return

In [5]:
# %pylab inline
# X_translations = []
# y_translations = []
# for i in xrange(1): # visualize this for one example
#     appendAllTranslations(X_train[i], y_train[i], X_translations, y_translations)

# # copied from examples 
# fig = plt.figure(figsize=(6, 6))  # figure size in inches
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# # plot the digits: each image is 8x8 pixels
# for i in xrange(len(X_translations)):
#     ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
#     ax.imshow(X_translations[i].reshape(28, 28), cmap=plt.cm.binary, interpolation='nearest')

In [6]:
print len(X_train)
print strftime("%Y-%m-%d %X"), "start"
X_translations = []
y_translations = []
for i in xrange(len(X_train)):
    if i % 100 == 0: print i,
    appendAllTranslations(X_train[i], y_train[i], X_translations, y_translations)
print
print strftime("%Y-%m-%d %X"), "end"
len(X_translations), len(y_translations)


42000
2015-05-03 11:02:03 start
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 

(3406147, 3406147)

In [10]:
print float(len(X_translations)) / len(X_train)

81.0987380952


In [7]:
# print strftime("%Y-%m-%d %X"), "PCA starting"
# pca65 = decomposition.RandomizedPCA(n_components=65, whiten=False)
# X_transform65 = pca65.fit_transform(X_translations)
# print strftime("%Y-%m-%d %X"), "PCA finished"
# knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
# knn1.fit(X_transform65, y_translations)
# print strftime("%Y-%m-%d %X"), "knn finished"
# # z = knn1.predict(pca65.transform(X_validate))
# # print metrics.accuracy_score(y_validate, z)
# # print strftime("%Y-%m-%d %X"), "finished scoring"

In [8]:
# z_test = knn1.predict(pca65.transform(X_test))
# print strftime("%Y-%m-%d %X"), "predicted test data"


In [9]:
# with open('pca_rf_with_translation_plus_shape_3_65.csv', 'w') as f:
#     f.write('ImageId,Label\n')
#     for i in xrange(28000):
#         f.write(str(i+1) + ',' + str(z_test[i]) + '\n')
